In [23]:
from glob import glob
import json
import re

pattern = r"\d+\.\s(.+)"
already = set()

files = glob('generate-questions/*')

questions = []
for f in files:
    with open(f) as fopen:
        d = json.load(fopen)
    keyword = d['q'][0]
    for q in re.findall(pattern, d['r']):
        if q in already:
            continue
        questions.append((q, keyword))
    
len(questions)

213261

In [24]:
import string

digits = set(string.digits)
rejected = ['\'', '"', 'http', '\n', '[', ']', '/', '`']

def contains_non_ascii(text):
    return any(ord(char) > 127 for char in text)

def reject_q(q):
    if q is None:
        return True
    if any([c in q for c in rejected]):
        return True
    if contains_non_ascii(q):
        return True
    if len(set(q) & digits):
        return True
    if len(q) < 20:
        return True
    if len(q) > 200:
        return True
    return False

In [25]:
from collections import defaultdict

filtered_q = defaultdict(list)
for q, k in questions:
    if len(q) < 10:
        continue
    if reject_q(q):
        continue
    
    filtered_q[k].append(q)
len(filtered_q)

58

In [31]:
len(filtered_q['d3 js'])

31

In [54]:
questions = []
for k, v in filtered_q.items():
    if len(v) < 100:
        continue
    v = sorted(v, key = lambda x: len(x), reverse = True)
    v = [(v_, k) for v_ in v][:1000]
    questions.extend(v)
    
len(questions)

57000

In [55]:
questions[0]

('Bolehkah anda menerangkan langkah-langkah yang diperlukan untuk membuat sistem pengurusan penyediaan semula dalam Go yang berfungsi dengan cekap dan tahan terhadap kegagalan?',
 'go distributed system')

In [56]:
with open('dedup-questions.json', 'w') as fopen:
    json.dump(questions, fopen)